# 2.5k Asset Backtest Example - SPO

## 1. Setup

#### 1.1 Load InvestOS module

In [1]:
# Add relative path to module lookup path...
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

# ... then import module
import investos as inv

# Import other required modules
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.plotting.backend = "plotly"
import numpy as np

#### 1.2 Load historical stock prices and volumes

Place in examples/data directory [data from Kaggle](https://www.kaggle.com/datasets/camnugent/sandp500/download?datasetVersionNumber=4)

Note: you will need to create (and sign into) a free Kaggle account

In [2]:
# A. Load S&P 500 tickers, names, and industries
dir_name = './data/'

df = pd.read_parquet(
    dir_name + '10k_synthetic.parquet'
)

#### 1.3 Split dfs into historical and forecast

In [3]:
import datetime

date_split_forecasts = datetime.date(2022, 1, 1)

df_forecast = df[df.date >= date_split_forecasts][['asset', 'date', 'return']]
df_actual = df

#### 1.4 Create (fake) forecasts

In [4]:
np.random.seed(0)

# Median (daily) return is VERY close to 0:
print("Median return:", df_forecast['return'].median())

# Cap returns at +- 10%...
df_forecast['return'] = df_forecast['return'].clip(-0.1, 0.1)

# ... then reduce signal...
df_forecast['return'] /= 10

# ... then add guassian noise to daily returns
std = df_actual[df_actual.date >= date_split_forecasts]['return'].var() ** 0.5
noise = np.random.normal(0, std, size=df_forecast.shape[0])

df_forecast['return'] = df_forecast['return'] + noise

Median return: 0.0002618778449523214


In [5]:
# Make sure predictions aren't too accurate:

agree_on_sign = np.sign(df_forecast['return']) == np.sign(
    df_actual[df_actual.date >= date_split_forecasts]['return']
)

print(
    "Return predictions have the right sign %.1f%% of the time" %
    ((agree_on_sign.sum() / agree_on_sign.shape[0]) * 100)
)

Return predictions have the right sign 53.2% of the time


## 2. Portfolio optimization

#### 2.1 Create portfolio optimization instance

In [6]:
from investos.portfolio_optimization.cost_model import *
from investos.portfolio_optimization.constraint_model import *
from investos.portfolio_optimization.risk_model import *

In [7]:
strategy = inv.portfolio_optimization.strategy.SPO(
    costs = [
        TradingCost() * 10, 
        HoldingCost(),
    ],
    constraints = [
        MaxLeverageConstraint(limit=3.0),
        MinWeightConstraint(), 
        MaxWeightConstraint(), 
        EqualLongShortConstraint() 
    ],
    risk_model = None,
)

backtest = inv.portfolio_optimization.Backtester(
    df_forecast,
    df_actual,
    strategy=strategy,
    start_date=datetime.date(2022, 1, 1),
    end_date=datetime.date(2022, 12, 30),
    aum=300_000_000,
)

In [8]:
# Started at 6:15 pm
# Ended at []
report = backtest.optimize()

Optimizing...


/opt/homebrew/lib/python3.9/site-packages/cvxpy/problems/problem.py:161: UserWarning: Constraint #3 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
/opt/homebrew/lib/python3.9/site-packages/cvxpy/problems/problem.py:161: UserWarning: Constraint #11 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "


Propagating for 2022-01-03
Propagating for 2022-01-04
Propagating for 2022-01-05
Propagating for 2022-01-06
Propagating for 2022-01-07
Propagating for 2022-01-10
Propagating for 2022-01-11
Propagating for 2022-01-12
Propagating for 2022-01-13
Propagating for 2022-01-14
Propagating for 2022-01-18
Propagating for 2022-01-19
Propagating for 2022-01-20
Propagating for 2022-01-21
Propagating for 2022-01-24
Propagating for 2022-01-25
Propagating for 2022-01-26
Propagating for 2022-01-27
Propagating for 2022-01-28
Propagating for 2022-01-31
Propagating for 2022-02-01
Propagating for 2022-02-02
Propagating for 2022-02-03
Propagating for 2022-02-04
Propagating for 2022-02-07
Propagating for 2022-02-08
Propagating for 2022-02-09
Propagating for 2022-02-10
Propagating for 2022-02-11


/opt/homebrew/lib/python3.9/site-packages/cvxpy/problems/problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Propagating for 2022-02-14
Propagating for 2022-02-15
Propagating for 2022-02-16
Propagating for 2022-02-17
Propagating for 2022-02-18
Propagating for 2022-02-22
Propagating for 2022-02-23
Propagating for 2022-02-24
Propagating for 2022-02-25
Propagating for 2022-02-28
Propagating for 2022-03-01
Propagating for 2022-03-02
Propagating for 2022-03-03
Propagating for 2022-03-04
Propagating for 2022-03-07
Propagating for 2022-03-08
Propagating for 2022-03-09
Propagating for 2022-03-10
Propagating for 2022-03-11
Propagating for 2022-03-14
Propagating for 2022-03-15
Propagating for 2022-03-16
Propagating for 2022-03-17
Propagating for 2022-03-18
Propagating for 2022-03-21
Propagating for 2022-03-22
Propagating for 2022-03-23
Propagating for 2022-03-24
Propagating for 2022-03-25
Propagating for 2022-03-28
Propagating for 2022-03-29
Propagating for 2022-03-30
Propagating for 2022-03-31
Propagating for 2022-04-01
Propagating for 2022-04-04
Propagating for 2022-04-05
Propagating for 2022-04-06
P

KeyboardInterrupt: 

#### 2.2 Backtest results

In [ ]:
fig = report.v.plot()
fig.layout.update(showlegend=False)
fig.show()

In [ ]:
report.summary

#### 2.3 Examples

**A) Default options:**

- Long short
- Equal long short
- 2x max leverage (i.e. exposure)
- 100MM AUM
- Default trading and holding cost models
- Default (long and short) position weight constraints [2.5%]

```python
# Code:

strategy = inv.portfolio_optimization.strategy.SPO()

backtest = inv.portfolio_optimization.Backtester(
    df_forecast,
    df_actual,
    strategy=strategy,
)
```

**B) Same as A, but 200MM AUM:**

```python
# Code:

strategy = inv.portfolio_optimization.strategy.SPO()

backtest = inv.portfolio_optimization.Backtester(
    df_forecast,
    df_actual,
    strategy=strategy,
    aum=200_000_000,
)
```

**C) Same as B, but half price movement on trading:**

```python
# Code:

strategy = inv.portfolio_optimization.strategy.SPO(
    costs = [
        TradingCost(price_movement_sensitivity=0.5), 
        HoldingCost(),
    ],
)

backtest = inv.portfolio_optimization.Backtester(
    df_forecast,
    df_actual,
    strategy=strategy,
    aum=200_000_000,
)
```

**D) Same as A, but long only (with no leverage):**

```python
# Code:

strategy = inv.portfolio_optimization.strategy.SPO(
    constraints = [
        LongOnlyConstraint(),
        MaxWeightConstraint(), 
    ],
)

backtest = inv.portfolio_optimization.Backtester(
    df_forecast,
    df_actual,
    strategy=strategy,
)
```

**E) Same as A, but 1x leverage:**

```python
# Code:

strategy = inv.portfolio_optimization.strategy.SPO(
    constraints = [
        MaxLeverageConstraint(limit=1.0),
        MinWeightConstraint(), 
        MaxWeightConstraint(), 
        EqualLongShortConstraint() 
    ],
)

backtest = inv.portfolio_optimization.Backtester(
    df_forecast,
    df_actual,
    strategy=strategy,
)
```

**F) Same as A, but discourage costly trades by 10x:**

```python
# Code:

strategy = inv.portfolio_optimization.strategy.SPO(
    costs = [
        TradingCost() * 10, 
        HoldingCost(),
    ],
)

backtest = inv.portfolio_optimization.Backtester(
    df_forecast,
    df_actual,
    strategy=strategy,
    start_date='2016-07-01',
    end_date='2018-01-01',
)
```